In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import StripePattern, ScrollZoomToggler, DualMap, FloatImage
import branca.colormap as cm
import random
import json
import warnings
import shapely
import os
import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [5]:
def reock_score(geom):
    geom_area = geom.geometry.area
    min_circle = geom.geometry.minimum_bounding_circle()
    min_circle_area = min_circle.area
    reock_score = geom_area / min_circle_area
    return reock_score

In [6]:
actual_results = pd.read_excel("../redistricting/2022_congressional_voting_results.xlsx")
actual_results["district"] = actual_results["district"].astype(str)
run = "moving_knife"

In [7]:
def dissolve_df(df, buffer_width=1e-4):
    dissolved = df.dissolve(by=f'DISTRICT_', aggfunc="sum").reset_index()
    dissolved = dissolved[['geometry', 'POP20', 'USH20_D', 'USH20_R']]
    dissolved['geometry'] = dissolved.buffer(buffer_width)
    dissolved['district'] = dissolved.index
    dissolved['district'] = dissolved['district'] + 1
    dissolved["Dem_PCT"] = round(
        dissolved['USH20_D']/(dissolved['USH20_D']+dissolved['USH20_R']), 4)*100
    return dissolved

In [8]:
state_dict = dict(
    az_2020={"districts": 9, "crs": "EPSG:2223"},
    ca_2020={"districts": 52, "crs": "EPSG:2225"},
    ct_2020={"districts": 5, "crs": "EPSG:2234"},
    fl_2020={"districts": 28, "crs": "EPSG:2225"},
    ga_2020={"districts": 14, "crs": "EPSG:2240"},
    ma_2020={"districts": 9, "crs": "EPSG:2249"},
    md_2020={"districts": 8, "crs": "EPSG:2248"},
    mi_2020={"districts": 13, "crs": "EPSG:2252"},
    mn_2020={"districts": 8, "crs": "EPSG:2811"},
    nc_2020={"districts": 14, "crs": "EPSG:2264"},
    nh_2020={"districts": 2, "crs": "EPSG:3437"},
    nj_2020={"districts": 12, "crs": "EPSG:2824"},
    nv_2020={"districts": 4, "crs": "EPSG:2821"},
    oh_2020={"districts": 15, "crs": "EPSG:2834"},
    or_2020={"districts": 6, "crs": "EPSG:2269"},
    pa_2020={"districts": 17, "crs": "EPSG:2271"},
    sc_2020={"districts": 7, "crs": "EPSG:2273"},
    tx_2020={"districts": 38, "crs": "EPSG:2277"},
    va_2020={"districts": 11, "crs": "EPSG:2283"},
    wi_2020={"districts": 8, "crs": "EPSG:2288"},
)

In [27]:
scores = []
for state in tqdm.tqdm(state_dict):
    state = state[:2].upper()

    state_results = actual_results.loc[actual_results["state"] == state]
    actual_districts = gpd.read_file( f"../data/{state.lower()}/district-shapes/POLYGON.shp")
    actual_districts["geometry"] = (actual_districts["geometry"].apply(shapely.validation.make_valid).buffer(1e-9))
    actual_districts = actual_districts.to_crs("EPSG:4326")

    

    algo_districts = gpd.read_file(f"../output/{state.lower()}_2020/{state.lower()}_2020_{run}.shp")
    algo_districts = algo_districts.to_crs("EPSG:4326")
    algo_districts["geometry"] = (algo_districts["geometry"].apply(shapely.validation.make_valid).buffer(1e-9))
    algo_districts = algo_districts.dissolve(by=f'DISTRICT_', aggfunc="sum").reset_index()
    algo_districts['geometry'] = algo_districts.buffer(1e-4)

    actual_reock_orig = np.mean(reock_score(actual_districts))
    algo_reock_orig = np.mean(reock_score(algo_districts))

    # Clipped Scores
    actual_districts_c = gpd.clip(actual_districts,algo_districts)
    actual_reock_clip = np.mean(reock_score(actual_districts_c))

    algo_districts_c = gpd.clip(algo_districts,actual_districts)
    algo_reock_clip = np.mean(reock_score(algo_districts_c))


    # Overall Reock Scores
    actual_reock_overall = reock_score(actual_districts.dissolve())[0]
    algo_reock_overall = reock_score(algo_districts.dissolve())[0]
    actual_reock_overall_c = reock_score(actual_districts_c.dissolve())[0]
    algo_reock_overall_c = reock_score(algo_districts_c.dissolve())[0]


    results = {
        "State": state,
        "Actual.Reock.Original": round(actual_reock_orig,5),
        "Algo.Reock.Original": round(algo_reock_orig,5),
        "Actual.Reock.Clipped": round(actual_reock_clip,5),
        "Algo.Reock.Clipped": round(algo_reock_clip,5),
        "Actual.Reock.Overall": round(actual_reock_overall,5),
        "Algo.Reock.Overall": round(algo_reock_overall,5),
        "Actual.Reock.Overall.Clipped": round(actual_reock_overall_c,5),
        "Algo.Reock.Overall.Clipped": round(algo_reock_overall_c,5),
    }
    
    

    scores.append(results)
    
scores_df = pd.DataFrame(scores)


100%|██████████| 20/20 [08:49<00:00, 26.46s/it]


In [28]:
scores_df

,State,Actual.Reock.Original,Algo.Reock.Original,Actual.Reock.Clipped,Algo.Reock.Clipped,Actual.Reock.Overall,Algo.Reock.Overall,Actual.Reock.Overall.Clipped,Algo.Reock.Overall.Clipped
0,AZ,0.39703,0.52583,0.39702,0.52578,0.64866,0.64870,0.64864,0.64864
1,CA,0.34625,0.26637,0.33204,0.26624,0.30405,0.29595,0.29577,0.29577
2,CT,0.43597,0.50309,0.43597,0.47415,0.40219,0.43518,0.40219,0.40219
3,FL,0.46675,0.32513,0.42770,0.32528,0.24927,0.22215,0.22246,0.22246
4,GA,0.45280,0.46625,0.45279,0.46688,0.52103,0.52109,0.52101,0.52101
5,MA,0.34002,0.40435,0.30312,0.40427,0.27608,0.22005,0.21999,0.21999
6,MD,0.32682,0.33549,0.30895,0.33541,0.19568,0.16082,0.16076,0.16076
7,MI,0.37816,0.40577,0.38557,0.40544,0.46521,0.28062,0.28056,0.28056
8,MN,0.40180,0.41704,0.39904,0.41700,0.45517,0.44247,0.44242,0.44242
9,NC,0.41814,0.44639,0.41814,0.44634,0.22200,0.22203,0.22200,0.22200


In [30]:
scores_df.to_csv(f"{run}_reock_scores.csv",index=False)

In [1]:
import pandas as pd

In [9]:
scores = []
for state in tqdm.tqdm(state_dict):
    info = gpd.read_file(f"../redistricting/data_prep/output/{state}")
    break

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]


In [10]:
info

,UNIQUE_ID,COUNTYFP,PCTNUM,PRECINCTNA,CDE_COUNTY,COUNTY_NAM,CON_DIST,SLDL_DIST,SLDU_DIST,USH20_D,USH20_R,POP20,geometry
0,02 ALPINE,001,AP0002,ALPINE,AP,Apache,01,07,07,105.0,271.0,664,"POLYGON ((1436786.757 973676.194, 1436758.529 ..."
1,03 CANYON DE CHELLY,001,AP0003,CANYON DE CHELLY,AP,Apache,01,07,07,1964.0,300.0,4666,"POLYGON ((1348618.816 1921826.442, 1348640.413..."
2,05 CHINLE,001,AP0005,CHINLE,AP,Apache,01,07,07,964.0,171.0,2949,"POLYGON ((1320446.726 1926158.469, 1321543.432..."
3,09 CONCHO,001,AP0009,CONCHO,AP,Apache,01,07,07,373.0,1407.0,2945,"POLYGON ((1416612.990 1263078.901, 1416776.975..."
4,11 COTTONWOOD,001,AP0011,COTTONWOOD,AP,Apache,01,07,07,747.0,89.0,1529,"POLYGON ((1319727.977 1879693.714, 1319593.114..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,003 CLIFTON 1,011,GN0003,CLIFTON 1,GN,Greenlee,01,14,14,381.0,356.0,2225,"POLYGON ((1508360.701 786149.967, 1508347.500 ..."
1485,006 EAST PLANTSITE,011,GN0006,EAST PLANTSITE,GN,Greenlee,01,14,14,59.0,142.0,849,"POLYGON ((1496040.125 756291.215, 1496160.364 ..."
1486,GHOST (232.00),025,YA0232,GHOST,YA,Yavapai,04,06,06,1241.0,1142.0,4063,"POLYGON ((662833.342 1382572.753, 662840.770 1..."
1487,BUG (222.00),025,YA0222,BUG,YA,Yavapai,04,01,01,814.0,2276.0,5826,"POLYGON ((827762.248 1091613.860, 827693.458 1..."
